In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


In [ ]:
train_data = pd.read_csv('../input/widsdatathon2021/TrainingWiDS2021.csv', index_col = 'encounter_id')

In [ ]:
test_data = pd.read_csv('../input/widsdatathon2021/UnlabeledWiDS2021.csv', index_col = 'encounter_id')

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.describe()

In [ ]:
train_data.drop(['Unnamed: 0', 'hospital_id', 'readmission_status', 'h1_glucose_max','h1_glucose_min'], axis = 1, inplace = True)
test_data.drop(['Unnamed: 0', 'hospital_id', 'readmission_status', 'h1_glucose_max','h1_glucose_min'], axis = 1, inplace = True)

In [ ]:
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        print(col)

In [ ]:
train_data['ethnicity'].unique()

In [ ]:
train_data['gender'].unique()

In [ ]:
train_data['hospital_admit_source'].unique()

In [ ]:
train_data['icu_admit_source'].unique()

In [ ]:
train_data['icu_stay_type'].unique()

In [ ]:
train_data['icu_type'].unique()

In [ ]:
object_col = ['ethnicity', 'gender' ,'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']

In [ ]:
for col in object_col:
    if train_data[col].isnull().sum() > 0:
        print(col)

In [ ]:
numeric_col = [col for col in train_data.columns if col not in object_col]      

In [ ]:
for col in numeric_col:
    if train_data[col].isnull().sum()>0:
        print(col)

In [ ]:
import seaborn as sns
sns.countplot(train_data['diabetes_mellitus'])

In [ ]:
y = train_data['diabetes_mellitus']
train_data.drop('diabetes_mellitus', axis = 1, inplace = True)

In [ ]:
for col in object_col:
    train_data[col] = train_data[col].fillna('MISS')
    test_data[col] = test_data[col].fillna('MISS')

In [ ]:
for col in numeric_col:
    train_data[col] = train_data[col].fillna(-999)
    test_data[col] = test_data[col].fillna(-999)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in object_col:
    train_data[col] = encoder.fit_transform(train_data[col])
    test_data[col] = encoder.transform(test_data[col])

In [ ]:
from sklearn.model_selection import train_test_split
train_x,valid_x, train_y, valid_y = train_test_split(train_data,y, test_size = 0.2, random_state = 0)

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from catboost import CatBoostClassifier
model_1 = CatBoostClassifier(iterations = 1500, learning_rate = 0.08, depth = 4)
model_1.fit(train_x, train_y)

In [ ]:
prediction_1 = model_1.predict_proba(valid_x)[:,1]

In [ ]:
prediction_0 = model_1.predict_proba(train_x)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(valid_y,prediction_1)

In [ ]:
test_prediction = model_4.predict_proba(test_data)[:,1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_2 = RandomForestClassifier(n_estimators= 500)
model_2.fit(train_x,train_y)

In [ ]:
prediction_2 = model_2.predict_proba(valid_x)[:,1]

In [ ]:
roc_auc_score(valid_y,prediction_2)

In [ ]:
from xgboost import XGBClassifier
model_3 = XGBClassifier(n_estimators = 500)
model_3.fit(train_x,train_y)

In [ ]:
prediction_3 = model_3.predict_proba(valid_x)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(valid_y, prediction_3)

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier
model_4 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.07)
model_4.fit(train_x,train_y)

In [ ]:
prediction_4 = model_4.predict_proba(valid_x)[:,1]

In [ ]:
roc_auc_score(valid_y, prediction_4)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
model_5 = AdaBoostClassifier()
model_5.fit(train_x, train_y)
prediction_5 = model_5.predict_proba(valid_x)[:,1]

In [ ]:
roc_auc_score(valid_y, prediction_5)

In [ ]:
new_train = train_data.reset_inde()
new_test = test_data.reset_index()

In [ ]:
new_train.drop('encounter_id', axis = 1, inplace = True)

In [ ]:
new_test.drop('encounter_id', axis = 1, inplace = True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
pred_test_full = 0
auc = []
i = 1

ensemble_model = VotingClassifier(estimators = [('first_boost', CatBoostClassifier(iterations = 1500, learning_rate = 0.08, 
                                                                                   depth = 4)),
                                                ('second_boost', CatBoostClassifier()),
                                               ('lightgbm', LGBMClassifier(n_estimators = 1000, learning_rate = 0.07))], voting = 'soft')
                                              

skf = StratifiedKFold(n_splits=25, shuffle=True)
for train_index,test_index in skf.split(new_train, y):
    X_train, X_test = new_train.iloc[train_index],new_train.iloc[test_index]
    Y_train, Y_test  = y.iloc[train_index], y.iloc[test_index]
    fit_model = ensemble_model.fit(X_train, Y_train)
    auc.append(roc_auc_score(Y_test, fit_model.predict(X_test)))
    pred_test = fit_model.predict_proba(new_test)[:,1]
    pred_test_full +=pred_test
    i+=1

In [ ]:
np.mean(auc)

In [ ]:
stratified_prediction = pred_test_full/25

In [ ]:
output = pd.DataFrame({
    'encounter_id': test_data.index,
    'diabetes_mellitus':stratified_prediction})

In [ ]:
output.to_csv('diabetes_27.csv', index = False)

In [ ]:
pd.read_csv('diabetes_27.csv')